In [29]:
import data_loader
import numpy as np
import pandas


# References
https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c

https://chartio.com/resources/tutorials/how-to-rename-columns-in-the-pandas-python-library/

https://towardsdatascience.com/handling-missing-values-in-machine-learning-part-2-222154b4b58e

https://stackoverflow.com/questions/45321406/missing-value-imputation-in-python-using-knn

https://stackoverflow.com/questions/44239269/fancyimpute-installation-in-anaconda

https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-py

# Data Loading

In [30]:
# load data and assign names
trdf, valdf = data_loader.load_train_data("data/adult.data", is_df=True)
## adding columns labels https://chartio.com/resources/tutorials/how-to-rename-columns-in-the-pandas-python-library/
trdf.columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country"
,"target"]
valdf.columns = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country"
,"target"]

## Replace '?' with np.nan
'?' in values represents missing data. So we replace it with np.nan

In [31]:
# missign values replaced as np.nan
trdf = trdf.replace(' ?',np.nan)
valdf = valdf.replace(' ?',np.nan)

## Check which features have missing values

In [32]:
trdf.isna().sum()

age                  0
workclass         1662
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1669
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     498
target               0
dtype: int64

### Observation
We observe that only categorical features have missing values. So we will use techniques to fill missing values for categorical features. First let's assign correct data type to categorical features

## Assign correct data type to categorical values

In [33]:
catcols = [1,3,5,6,7,8,9,13,14] # list of categorical features
trdf.iloc[:,catcols] = trdf.iloc[:,catcols].astype('category')

In [34]:
trdf['workclass'].cat.categories ## here we verify that nan is not treated as a category

Index([' Federal-gov', ' Local-gov', ' Never-worked', ' Private',
       ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
      dtype='object')

Now we will impute missing values to nan using k-nearest neighbor imputation technique described [here](https://towardsdatascience.com/handling-missing-values-in-machine-learning-part-2-222154b4b58e)

## Resolve Missing values 


### k-NN Imputation
For using kNN imputation, we will use fancyimpute library because sklearn only supports mean, median, mode impute as shown [here](https://stackoverflow.com/questions/45321406/missing-value-imputation-in-python-using-knn)

In order to install fancyimpute we execute the following commands on the terminal as shown [here](https://stackoverflow.com/questions/44239269/fancyimpute-installation-in-anaconda)

``` bash
conda install ecos  
conda install CVXcanon  
pip install fancyimpute  

```

We were getting errors in install fancyimpute and decided to use IterativeImputer from sklearn. [Here](https://scikit-learn.org/dev/auto_examples/impute/plot_missing_values.html#sphx-glr-auto-examples-impute-plot-missing-values-py) is an official example for using IterativeImputer

In [53]:
#from sklearn.experimental import enable_iterative_imputer  # noq
#from sklearn.impute import IterativeImputer

### Treat nan as a separate category

## Give category codes to each value in every feature

## Data Exploration

## Covariance of features - Principal component analysis to remove redundant features
[covariance finding](https://towardsdatascience.com/handling-missing-values-in-machine-learning-part-2-222154b4b58e)

## Analyze features with missing values

### Missing values in categorical data

# Models

## Naive Bayes - Baseline model


[Selecting subsets of data](https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c)

TODO: missing values

In [36]:
# trdf.dropna(inplace=True)
# catcols = [1,3,5,6,7,8,9,13,14]
# trdf[catcols] = trdf[catcols].astype('category')
trdf.iloc[:,catcols] = trdf.iloc[:,catcols].apply(lambda x: x.cat.codes)
# trdf
# trdf[1].cat.codes
trdf.iloc[:,14].shape

(29315,)

In [37]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
learner = model.fit(trdf.iloc[:,0:14], trdf.iloc[:,14])


In [38]:
catcols = [1,3,5,6,7,8,9,13,14]
valdf.iloc[:,catcols] = valdf.iloc[:,catcols].astype('category')
valdf.iloc[:,catcols] = valdf.iloc[:,catcols].apply(lambda x: x.cat.codes)
# trdf
# trdf[1].cat.codes
valdf.shape

(3246, 15)

In [39]:
learner.predict(valdf.iloc[:,0:14])

array([0, 1, 1, ..., 0, 0, 1], dtype=int8)

In [40]:
learner.score(valdf.iloc[:,0:14], valdf.iloc[:,14])

0.8009858287122612

## kNN


### 1 nearest

In [41]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=29)
learner = model.fit(trdf.iloc[:,0:14], trdf.iloc[:,14])
learner.predict(valdf.iloc[:,0:14])

array([0, 1, 1, ..., 0, 0, 1], dtype=int8)

In [42]:
learner.score(valdf.iloc[:,0:14], valdf.iloc[:,14])

0.8034504004929144

### 5 nearest

## ID3

### Parameterization

## NbTree
[doc](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-py)

In [52]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
learner = model.fit(trdf.iloc[:,0:14], trdf.iloc[:,14])
learner.predict(valdf.iloc[:,0:14])
learner.score(valdf.iloc[:,0:14], valdf.iloc[:,14])

/home/clyton/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8357979051139864

### Parameterization

In [44]:
# from sklearn.svm import SVC
# svc = SVC(C=1.0, kernel = "linear")
# svc.fit(trdf.iloc[:,0:14], trdf.iloc[:,14])
# svc.predict(valdf.iloc[:,0:14])

In [45]:
# svc.score(valdf.iloc[:,0:14], valdf.iloc[:,14])

## Neural Network
### Parameterization

In [46]:
import sklearn.neural_network.multilayer_perceptron as mlp
model = mlp.MLPClassifier()
learner = model.fit(trdf.iloc[:,0:14], trdf.iloc[:,14])

In [47]:
learner.predict(valdf.iloc[:,0:14])

array([0, 0, 1, ..., 0, 0, 0], dtype=int8)

In [48]:
learner.score(valdf.iloc[:,0:14],  valdf.iloc[:,14])

0.7732593961799138

## XGBoost
### Parameterization

In [49]:
import xgboost as xgb

In [50]:
model = xgb.XGBClassifier()
learner = model.fit(trdf.iloc[:,0:14], trdf.iloc[:,14])


In [51]:
learner.score(valdf.iloc[:,0:14],  valdf.iloc[:,14])

0.8585951940850277